In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, json
from scipy.optimize import curve_fit
from scipy.stats import pearsonr, spearmanr

import networkx as nx

import seaborn as sns

this script finds all the hub pairs within 50kb in all chr in K562 (all 26,148 hubs)

1. find all hub pairs

2. calc scaling factors

3. set a cutoff and test lossely connected hub pairs on chr, find the proportion


In [ ]:
# load 26148 hub dt
dt = pd.read_csv('resources/all_hubs.txt', sep='\t')
dt = dt[['c','s']]
dt

,c,s
0,chr1,87090000
1,chr1,106485000
2,chr1,109015000
3,chr1,120635000
4,chr1,121165000
...,...,...
26143,chrX,146360000
26144,chrX,149100000
26145,chrX,146740000
26146,chrX,147250000


In [ ]:
# first, remove sites in the blk list or centromere
# black-list data can be obtained from https://github.com/Boyle-Lab/Blacklist/blob/master/lists/Blacklist_v1/hg19-blacklist.bed.gz
blk_dt = pd.read_csv('path/to/blk/blacklist.bed6format.bed', sep='\t', header=None, )
blk_dt = blk_dt[[0,1,2]]
blk_dt.columns = [0,1,2]
# print(blk_dt.iloc[:5, ])

# gap.txt.gz can be obtained in the folder https://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/
gap_dt = pd.read_csv('path/to/gap/gap.txt', sep='\t', header=None,)
gap_dt = gap_dt[[1,2,3,7,8]]
gap_dt = gap_dt[
    (gap_dt[8]=='no') &
    (gap_dt[7]=='centromere')
]
gap_dt = gap_dt[[1,2,3]]
gap_dt.columns = [0,1,2]
# print(gap_dt.iloc[:5, ])

# transloc site specifically in K562 (hg19), we also remove 500k within the transloc site
trans_dt = pd.DataFrame(
    {0: ['chr9', 'chr22'],
     1: [133089268, 24022552],
     2: [133263062, 24160224]}
)
# print(trans_dt)


rm_dt = pd.concat((blk_dt, gap_dt, trans_dt), ignore_index=True)
rm_dt['s_int'] = (rm_dt[1]//5000*5000).astype(int)
rm_dt['e_int'] = (rm_dt[2]//5000*5000+50000).astype(int)
rm_dt

chr_num_list = ['chr{}'.format(i) for i in range(1,23)] + ['chrX']


In [ ]:
# load hic file and build graphs
new_dt_chr = {}
graph_dt_chr = {}

for chr_num in chr_num_list:
    # if chr_num=='chr2':
    #     break
    rm_dt_thischr = rm_dt[rm_dt[0]==chr_num].copy()
    # print(rm_dt_thischr)
    rm_loci_list = []
    for s,e in rm_dt_thischr.iloc[:,3:].values:
        # print(s,e)
        rm_loci_list.extend([i for i in range(s,e,5000)])
    rm_loci_list = list(set(rm_loci_list))

    dt_remain = dt[dt['c']==chr_num]
    dt_remain = dt_remain[~dt_remain['s'].isin(rm_loci_list)]
    # print(dt_remain)

    if chr_num=='chr9':
        new_dt_chr['chr9_before'] = dt_remain[dt_remain['s']<133089268].reset_index(drop=True).copy()
        new_dt_chr['chr9_after'] = dt_remain[dt_remain['s']>133263062].reset_index(drop=True).copy()
        prob_dt = pd.read_csv('path/to/save_folder/chr9_K562_prob.5000.txt', sep='\t', header=None)
    elif chr_num=='chr22':
        new_dt_chr['chr22_before'] = dt_remain[dt_remain['s']<24022552].reset_index(drop=True).copy()
        new_dt_chr['chr22_after'] = dt_remain[dt_remain['s']>24160224].reset_index(drop=True).copy()
        prob_dt = pd.read_csv('path/to/save_folder/chr22_K562_prob.5000.txt', sep='\t', header=None)
    else:
        new_dt_chr[chr_num] = dt_remain.reset_index(drop=True).copy()
        prob_dt = pd.read_csv('path/to/save_folder/{}_K562_prob.5000.txt'.format(chr_num), sep='\t', header=None)

    prob_dt = prob_dt[prob_dt[4]<=-20]
    prob_dt = prob_dt[~(prob_dt[0].isin(rm_loci_list) | prob_dt[1].isin(rm_loci_list))]

    G = nx.from_pandas_edgelist(prob_dt, 0, 1, create_using=nx.Graph())
    graph_dt_chr[chr_num] = G

graph_dt_chr['chr9_before'] = graph_dt_chr['chr9']
graph_dt_chr['chr9_after'] = graph_dt_chr['chr9']
graph_dt_chr['chr22_before'] = graph_dt_chr['chr22']
graph_dt_chr['chr22_after'] = graph_dt_chr['chr22']




In [ ]:
# find hub pair in this part
def find_hub_pairs(chrid, dt, G, max_dist=50000):


    degree_dict = dict(G.degree())
    dt_ = dt[dt['s'].isin(degree_dict.keys())].copy()

    s1_hist = []
    s2_hist = []
    dist_hist = []
    d1_hist = []
    d2_hist = []
    path_1_hist = []
    path_2_hist = []
    path_3_hist = []
    path_4_hist = []


    # use two pointer method
    srt_list = sorted(list(dt_['s'])) # hub start coord sorted
    print(srt_list)
    left = 0
    right = 1

    while left < len(srt_list) - 1 and right < len(srt_list):
        if left == right:
            right += 1
        s1 = srt_list[left]
        s2 = srt_list[right]
        diff = s2 - s1

        if diff <= max_dist:
            s1_hist.append(s1)
            s2_hist.append(s2)
            dist_hist.append(diff)
            d1_hist.append(degree_dict[s1])
            d2_hist.append(degree_dict[s2])

            print('{}-{}'.format(s1, s2))
            print('{}-{}'.format(left, right))
            cnt=0
            for path in nx.all_simple_paths(G, s1, s2, cutoff=1):
                if len(path)>1:
                    cnt += 1
            path_1_hist.append(cnt)

            cnt=0
            for path in nx.all_simple_paths(G, s1, s2, cutoff=2):
                if len(path)>2:
                    cnt += 1
            path_2_hist.append(cnt)

            cnt=0
            for path in nx.all_simple_paths(G, s1, s2, cutoff=3):
                if len(path)>3:
                    cnt += 1
            path_3_hist.append(cnt)

            cnt=0
            for path in nx.all_simple_paths(G, s1, s2, cutoff=4):
                if len(path)>4:
                    cnt += 1
            path_4_hist.append(cnt)

            right += 1

        else:
            left += 1
            right = left + 1


    results = pd.DataFrame({
        's1':s1_hist,
        's2':s2_hist,
        'dist':dist_hist,
        'd1':d1_hist,
        'd2':d2_hist,
        'path_1':path_1_hist,
        'path_2':path_2_hist,
        'path_3':path_3_hist,
        'path_4':path_4_hist,

    })
    results['c'] = chrid

    return results



# for fname in os.listdir(base_folder):
#     # print('----------')
#     # print(fname)
#     c,coord_str,dist_str = fname.split('.')
#     s1, s2 = coord_str.split('_')
#     s1, s2 = int(s1), int(s2)

#     if (c in rm_dict):
#         if (s1 in rm_dict[c]) or (s2 in rm_dict[c]):
#             print('ERROR, HUB REMOVED!')
#             continue

#     dist = int(dist_str[4:])
#     # print(c,s1,s2,dist)
#     # load data summary json
#     if not os.path.exists(os.path.join(base_folder, fname, 'data_summary_dict.json')):
#         continue
#     with open(os.path.join(base_folder, fname, 'data_summary_dict.json'), 'r') as f:
#         dt = json.load(f)
#     # print(dt)
#     s1_hist.append(s1)
#     s2_hist.append(s2)
#     chr_hist.append(c)
#     dist_hist.append(dist)
#     d1_hist.append(dt['degree.1'])
#     d2_hist.append(dt['degree.2'])
#     path_1_hist.append(dt['number_shortest_path.cutoff_1'])
#     path_2_hist.append(dt['number_shortest_path.cutoff_2'])
#     path_3_hist.append(dt['number_shortest_path.cutoff_3'])
#     path_4_hist.append(dt['number_shortest_path.cutoff_4'])


# result_df_ess_vs_ess = pd.DataFrame({
#     's1':s1_hist,
#     's2':s2_hist,
#     'chr':chr_hist,
#     'dist':dist_hist,
#     'degree.1':d1_hist,
#     'degree.2':d2_hist,
#     'path_step.1':path_1_hist,
#     'path_step.2':path_2_hist,
#     'path_step.3':path_3_hist,
#     'path_step.4':path_4_hist,
# })
# result_df_ess_vs_ess
# result_df_ess_vs_ess = result_df_ess_vs_ess.sort_values(['chr', 'dist', 's1']).copy().reset_index(drop=True)
# # because ess hubs are too far away, not closely connected
# result_df_ess_vs_ess


results = find_hub_pairs('chr1', new_dt_chr['chr1'], graph_dt_chr['chr1']) # test code



scaling factor analysis

In [ ]:
# normalize scaling factors
dt_all['deg_geom'] = (dt_all['degree.1'] * dt_all['degree.2']) ** (1/2)
dt_all['s1_norm'] = dt_all['path_step.1'] / dt_all['deg_geom']
dt_all['s2_norm'] = dt_all['path_step.2'] / dt_all['deg_geom']
dt_all['s3_norm'] = dt_all['path_step.3'] / dt_all['deg_geom']
dt_all['s4_norm'] = dt_all['path_step.4'] / dt_all['deg_geom']
dt_all

dt_all = dt_all.sort_values('chr').reset_index(drop=True)
dt_all

In [ ]:
# curve fit
def exp_curve(x, A, C):
	return A * np.exp(4*x) + C

for chrid, dt_ in dt_all.groupby('chr'):
    print(chrid)
    for i in range(len(dt_)):
        # try:
        (a, c), _ = curve_fit(
                        exp_curve, [1,2,3,4], list(dt_.iloc[i, 12:16]),
                        # [1, 1, np.median(cycles), 0], # initial guess for all params, very important
                        )
        # except:
        #     # try:
        #     #     popt, _ = curve_fit(
        #     #         exp_curve, [1,2,3,4], list(result_df_chr11.iloc[i, 12:16]),
        #     #         [1e-5, 0.00001, 1e-2], # initial guess for all params, very important
        #     #         )
        #     # except:
        #     print('fail')
        #     continue
        # print(result_df_chr11.iloc[i,0])
        print(a,c)

        plt.plot([1,2,3,4], list(dt_.iloc[i, 12:16]))
        plt.text(4.2, list(dt_.iloc[i, 12:16])[-1], '{},    {}'.format(a,c))
    plt.show()